In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install scikit-surprise
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
import pandas as pd
from collections import defaultdict
from operator import itemgetter
import heapq
import pandas as pd
import numpy as np


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/deviants_pg5/complted data/preprocessed_data/movies_small.csv')

In [ ]:
def load_dataset():
    reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
    ratings_dataset = Dataset.load_from_file('/content/drive/MyDrive/deviants_pg5/complted data/preprocessed_data/ratings_small.csv', reader=reader)
    return (ratings_dataset)
dataset = load_dataset()

# Build a full Surprise training set from dataset
trainset = dataset.build_full_trainset()

##**Similarity Matrix**

####**KNNBasic** is basic collaborative filtering algorithm that uses KNN algorithm in predicting the recommendations. We use cosine similarity as the similarity score criterion.

####user_based or not? Whether similarities would be computed between users or between items. This has a huge impact on the performance of a prediction algorithm.


In [ ]:
similarity_matrix = KNNBasic(sim_options={
        'name': 'cosine',
        'user_based': True
        })\
        .fit(trainset)\
        .compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# Pick a random user ID, has to be a numeric string.
test_subject = '6'

# Get the top K items user rated
k = 20


In [ ]:
# When using Surprise, there are RAW and INNER IDs.
# Raw IDs are the IDs, strings or numbers, you use when
# creating the trainset. The raw ID will be converted to
# an unique integer Surprise can more easily manipulate
# for computations.
#
# So in order to find an user inside the trainset, you
# need to convert their RAW ID to the INNER Id. Read
# Reference for more info https://surprise.readthedocs.io/en/stable/FAQ.html#what-are-raw-and-inner-ids
test_subject_iid = trainset.to_inner_uid(test_subject)

# Get the top K items we rated
test_subject_ratings = trainset.ur[test_subject_iid]
k_neighbors = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])

In [ ]:
# Default dict is basically a standard dictionary,
# the difference beeing that it doesn't throw an error
# when trying to access a key which does not exist,
# instead a new entry, with that key, is created.
candidates = defaultdict(float)

for itemID, rating in k_neighbors:
    try:
      similaritities = similarity_matrix[itemID]
      for innerID, score in enumerate(similaritities):
          candidates[innerID] += score * (rating / 5.0)
    except:
      continue

In [ ]:
# Build a dictionary of movies the user has watched
watched = {}
for itemID, rating in trainset.ur[test_subject_iid]:
  watched[itemID] = 1

# Add items to list of user's recommendations
# If they are similar to their favorite movies,
# AND have not already been watched.
recommendations = []

position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
  if not itemID in watched:
    recommendations.append(trainset.to_raw_iid(itemID))
    position += 1
    if (position > 10): break # We only want top 10


In [ ]:
recom = pd.DataFrame(recommendations,columns=['movieId'],dtype=int)

In [ ]:
recom.astype(int)
pd.merge(df,recom,on='movieId')['title']

0                                         Crumb (1994)
1                                 Grifters, The (1990)
2                           Black Cauldron, The (1985)
3                                            Go (1999)
4     Star Wars: Episode I - The Phantom Menace (1999)
5         Austin Powers: The Spy Who Shagged Me (1999)
6                              Dirty Dozen, The (1967)
7                                      Magnolia (1999)
8                                 Wayne's World (1992)
9                                 Thirteen Days (2000)
10                        Louis C.K.: Hilarious (2010)
Name: title, dtype: object